In [1]:
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [2]:
df =  pd.read_csv("/Users/MattNwerem/Downloads/COVID_Kaggle_CORD-19/metadata.csv")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,13,14,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df["abstract"].value_counts()

Unknown                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [4]:
df.head(5)

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636.0,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967.0,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972.0,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871.0,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888.0,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [5]:
df["abstract"][0] #test

'OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (35%) with upper respiratory tract infections, and 2 (5%) with bronchiolitis. Cough (82.5%), fever (75%), and malaise (58.8%) were 

In [6]:
df["abstract"][1] #test

'Inflammatory diseases of the respiratory tract are commonly associated with elevated production of nitric oxide (NO•) and increased indices of NO• -dependent oxidative stress. Although NO• is known to have anti-microbial, anti-inflammatory and anti-oxidant properties, various lines of evidence support the contribution of NO• to lung injury in several disease models. On the basis of biochemical evidence, it is often presumed that such NO• -dependent oxidations are due to the formation of the oxidant peroxynitrite, although alternative mechanisms involving the phagocyte-derived heme proteins myeloperoxidase and eosinophil peroxidase might be operative during conditions of inflammation. Because of the overwhelming literature on NO• generation and activities in the respiratory tract, it would be beyond the scope of this commentary to review this area comprehensively. Instead, it focuses on recent evidence and concepts of the presumed contribution of NO• to inflammatory diseases of the lun

In [7]:
#Testing what data looks like
'''
i = 0
for s in df["abstract"]:
    print(s)
    i+=1
    if(i == 5):
        break
'''

'\ni = 0\nfor s in df["abstract"]:\n    print(s)\n    i+=1\n    if(i == 5):\n        break\n'

In [8]:
df.dtypes

cord_uid             object
sha                  object
source_x             object
title                object
doi                  object
pmcid                object
pubmed_id           float64
license              object
abstract             object
publish_time         object
authors              object
journal              object
mag_id              float64
who_covidence_id     object
arxiv_id             object
pdf_json_files       object
pmc_json_files       object
url                  object
s2_id               float64
dtype: object

In [9]:
new_dtypes = {"title": str, "abstract": str} #these dtypes must be changed in order to use sent_tokenize()
df = df.astype(new_dtypes)

In [10]:
print(df.dtypes) #check to see if dtpypes changed

cord_uid             object
sha                  object
source_x             object
title                object
doi                  object
pmcid                object
pubmed_id           float64
license              object
abstract             object
publish_time         object
authors              object
journal              object
mag_id              float64
who_covidence_id     object
arxiv_id             object
pdf_json_files       object
pmc_json_files       object
url                  object
s2_id               float64
dtype: object


In [28]:
sentences = []
words = []

for s in df['abstract']:
  replace = s.replace("[^a-zA-Z]", " ")
  clean_sentence = replace.lower()
  sentences.append(sent_tokenize(clean_sentence))

In [29]:
sentences = [y for x in sentences for y in x] ## literally all text from all articles

In [30]:
for s in df['abstract']:
    replace = s.replace("[^a-zA-Z]", " ")
    clean_sentence = replace.lower()
    words.append(word_tokenize(clean_sentence))

In [31]:
len(df['abstract'])

158947

In [32]:
len(words)

158947

In [33]:
from nltk.corpus import stopwords

In [34]:
#Filtering out stopwords
stop_words = set(stopwords.words("english"))
newStopWords = [',','.','(',')','%',':',';']
stop_words.update(newStopWords)

non_filtered_words = []

stop_words  = list(stop_words) 
print("Stop Words: ", stop_words)


for w in words:
    if w in stop_words:
        continue
    else:
        non_filtered_words.append(w)

filtered_words = []

for part in non_filtered_words:
    for words in part:
        if words not in stop_words:
            filtered_words.append(words)
print("Filtered Word Length:",len(filtered_words))

Stop Words:  ['m', 'such', "doesn't", 'mightn', ')', "you'll", 'yourselves', 'out', 'yours', 'until', 'same', 'over', 'him', 'other', 'been', 'down', 'couldn', 'but', 'just', 'we', 'itself', 'as', 'being', 'can', 'very', 's', 'its', 'd', "hadn't", 'herself', 'myself', "don't", 'above', 'these', 'off', 'ma', 'wouldn', 'at', 'yourself', 'below', 'each', 'or', 'of', 'ours', "you've", 'again', 'doesn', 'with', 'not', 'hasn', '(', "weren't", "needn't", 'that', 'after', 'there', 'any', 'did', "hasn't", 'into', 'too', 'o', 'about', 'during', "mustn't", 'both', 'are', 'few', 'be', 'more', "wasn't", ';', 'whom', 'do', 'his', 'having', 'those', 'further', 'why', 'will', 're', "won't", "should've", 'does', "you're", 'only', "wouldn't", "shouldn't", 'were', 'then', 'no', 'was', 'an', 'had', 'where', 'my', 'a', 'if', 't', 'and', 'so', 'shan', 'himself', "you'd", "shan't", 'has', "couldn't", 'should', 'through', 'most', ',', 'up', 've', 'this', 'wasn', ':', "isn't", 'from', 'didn', "haven't", 'their

In [35]:
print(filtered_words[:1000]) # first 1000 words

['objective', 'retrospective', 'chart', 'review', 'describes', 'epidemiology', 'clinical', 'features', '40', 'patients', 'culture-proven', 'mycoplasma', 'pneumoniae', 'infections', 'king', 'abdulaziz', 'university', 'hospital', 'jeddah', 'saudi', 'arabia', 'methods', 'patients', 'positive', 'm.', 'pneumoniae', 'cultures', 'respiratory', 'specimens', 'january', '1997', 'december', '1998', 'identified', 'microbiology', 'records', 'charts', 'patients', 'reviewed', 'results', '40', 'patients', 'identified', '33', '82.5', 'required', 'admission', 'infections', '92.5', 'community-acquired', 'infection', 'affected', 'age', 'groups', 'common', 'infants', '32.5', 'pre-school', 'children', '22.5', 'occurred', 'year-round', 'common', 'fall', '35', 'spring', '30', 'three-quarters', 'patients', '77.5', 'comorbidities', 'twenty-four', 'isolates', '60', 'associated', 'pneumonia', '14', '35', 'upper', 'respiratory', 'tract', 'infections', '2', '5', 'bronchiolitis', 'cough', '82.5', 'fever', '75', 'mal

In [36]:
ps = PorterStemmer()
stemmed_words = []

#for words in filtered_words:
#    sample_w = ps.stem(words)
#    stemmed_words.append(sample_w)

In [37]:
def create_frequency_table(list1) -> dict:

    freqTable = dict()
    for word in list1:
        if word in freqTable:
            freqTable[word] += 1
        else:
            freqTable[word] = 1

    return freqTable

In [38]:
freqTable = create_frequency_table(filtered_words)

In [39]:
freqTable

{'objective': 10298,
 'retrospective': 4490,
 'chart': 386,
 'review': 19579,
 'describes': 2034,
 'epidemiology': 3061,
 'clinical': 44703,
 'features': 6415,
 '40': 3480,
 'patients': 131641,
 'culture-proven': 8,
 'mycoplasma': 856,
 'pneumoniae': 1884,
 'infections': 22698,
 'king': 186,
 'abdulaziz': 22,
 'university': 2990,
 'hospital': 16439,
 'jeddah': 91,
 'saudi': 1143,
 'arabia': 930,
 'methods': 36538,
 'positive': 15850,
 'm.': 1779,
 'cultures': 1994,
 'respiratory': 48768,
 'specimens': 4714,
 'january': 5709,
 '1997': 268,
 'december': 4816,
 '1998': 274,
 'identified': 18131,
 'microbiology': 560,
 'records': 2025,
 'charts': 210,
 'reviewed': 4129,
 'results': 58843,
 '33': 1791,
 '82.5': 47,
 'required': 8377,
 'admission': 5051,
 '92.5': 53,
 'community-acquired': 899,
 'infection': 56463,
 'affected': 5884,
 'age': 16769,
 'groups': 15243,
 'common': 13873,
 'infants': 3978,
 '32.5': 83,
 'pre-school': 25,
 'children': 16223,
 '22.5': 94,
 'occurred': 6220,
 'year-

In [40]:
sort_orders = sorted(freqTable.items(), key=lambda x: x[1], reverse=True)

In [41]:
sort_orders

[('patients', 131641),
 ('covid-19', 80271),
 ('virus', 62451),
 ('disease', 59064),
 ('results', 58843),
 ('study', 56647),
 ('infection', 56463),
 ('respiratory', 48768),
 ('health', 46065),
 ('clinical', 44703),
 ('nan', 41830),
 ('viral', 40505),
 ('data', 39301),
 ('using', 38585),
 ('may', 38053),
 ('coronavirus', 37644),
 ('cases', 37514),
 ('=', 36609),
 ('methods', 36538),
 ('p', 36225),
 ('treatment', 35859),
 ('cells', 34528),
 ('also', 33537),
 ('2', 32768),
 ('used', 32225),
 ('group', 30501),
 ('risk', 30413),
 ('studies', 29168),
 ('severe', 29168),
 ('viruses', 28845),
 ('two', 28773),
 ('acute', 28170),
 ('pandemic', 28152),
 ('human', 27354),
 ('associated', 27308),
 ('protein', 26961),
 ('care', 26832),
 ('time', 26602),
 ('analysis', 26537),
 ('sars-cov-2', 26032),
 ('1', 25950),
 ('one', 25772),
 ('use', 24956),
 ('compared', 24106),
 ('however', 23739),
 ('new', 23738),
 ('control', 23363),
 ('infections', 22698),
 ('different', 22580),
 ('including', 22356),
 ('h